In [1]:
import pandas as pd 
import os
import numpy as np 
import sys
sys.path.append('./../..')
sys.path.append('./..')
from tqdm import tqdm
import multiprocessing
from sklearn.preprocessing import StandardScaler
from itertools import combinations
from collections import Counter
from pandarallel import pandarallel
pandarallel.initialize()
from scipy.io import arff
import pandas as pd
from collections import Counter
from tqdm import tqdm 
import numpy as np
from sklearn.model_selection import train_test_split
from common_utils import utils 

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
data = arff.loadarff('gureKddcup6percent.arff')


In [3]:
df = pd.DataFrame(data[0])

In [4]:
df.head(5)

connection_number    start_time  orig_port  resp_port            orig_ip  \
0         75017989.0  9.006612e+08    10329.0       80.0  b'172.16.113.204'   
1         45025053.0  8.988652e+08     1107.0       53.0    b'192.168.1.10'   
2         72058899.0  9.004260e+08       20.0    18932.0  b'172.16.114.148'   
3         64551576.0  8.999874e+08    13682.0       80.0  b'172.16.116.201'   
4         11016178.0  8.966895e+08    10369.0       80.0  b'172.16.116.194'   

             resp_ip  duration protocol_type    service   flag  ...  \
0  b'192.215.120.11'  0.016895        b'tcp'    b'http'  b'SF'  ...   
1   b'172.16.112.20'  0.000000        b'udp'  b'domain'  b'S0'  ...   
2    b'135.8.60.182'  0.025245        b'tcp'   b'other'  b'SF'  ...   
3    b'207.25.71.28'  0.274966        b'tcp'    b'http'  b'SF'  ...   
4   b'140.175.21.20'  0.060163        b'tcp'    b'http'  b'SF'  ...   

   dst_host_srv_count  dst_host_same_srv_rate dst_host_diff_srv_rate  \
0                96.0                     1.0                    0.0   
1                38.0                     1.0                    0.0   
2                 0.0                     0.0                    1.0   
3                57.0                     1.0                    0.0   
4                84.0                     1.0                    0.0   

   dst_host_same_src_port_rate  dst_host_srv_diff_host_rate  \
0                         0.00                     0.729167   
1                         0.00                     0.394737   
2                         0.12                     0.000000   
3                         0.00                     0.684211   
4                         0.00                     0.547619   

   dst_host_serror_rate  dst_host_srv_serror_rate dst_host_rerror_rate  \
0              0.000000                  0.000000                  0.0   
1              0.652174                  0.789474                  0.0   
2              0.000000                  0.000000                  0.0   
3              0.111111                  0.087719                  0.0   
4              0.000000                  0.000000                  0.0   

   dst_host_srv_error_rate      class  
0                      0.0  b'normal'  
1                      0.0  b'normal'  
2                      0.0  b'normal'  
3                      0.0  b'normal'  
4                      0.0  b'normal'  

[5 rows x 48 columns]

In [5]:
attributes_file = 'attributes.txt'
df_attributes = pd.read_csv(attributes_file, delimiter=';',header=None)

In [6]:
df_attributes

0  \
0                      duration   
1                 protocol_type   
2                       service   
3                          flag   
4                     src_bytes   
5                     dst_bytes   
6                          land   
7                wrong_fragment   
8                        urgent   
9                           hot   
10            num_failed_logins   
11                    logged_in   
12              num_compromised   
13                   root_shell   
14                 su_attempted   
15                     num_root   
16           num_file_creations   
17                   num_shells   
18             num_access_files   
19            num_outbound_cmds   
20                 is_hot_login   
21               is_guest_login   
22                        count   
23                    srv_count   
24                  serror_rate   
25              srv_serror_rate   
26                  rerror_rate   
27              srv_rerror_rate   
28                same_srv_rate   
29                diff_srv_rate   
30           srv_diff_host_rate   
31               dst_host_count   
32           dst_host_srv_count   
33       dst_host_same_srv_rate   
34       dst_host_diff_srv_rate   
35  dst_host_same_src_port_rate   
36  dst_host_srv_diff_host_rate   
37         dst_host_serror_rate   
38     dst_host_srv_serror_rate   
39         dst_host_rerror_rate   
40      dst_host_srv_error_rate   
41                        class   

                                                    1  
0                                             numeric  
1                                      {tcp,udp,icmp}  
2   {telnet,finger,ftp,other,ftp_data,login,shell,...  
3   {SF,S2,S1,S3,OTH,REJ,RSTO,S0,RSTR,RSTOS0,SH,RS...  
4                                             numeric  
5                                             numeric  
6                                               {0,1}  
7                                             numeric  
8                                             numeric  
9                                             numeric  
10                                            numeric  
11                                              {0,1}  
12                                            numeric  
13                                              {0,1}  
14                                            numeric  
15                                            numeric  
16                                            numeric  
17                                            numeric  
18                                            numeric  
19                                            numeric  
20                                              {0,1}  
21                                              {0,1}  
22                                            numeric  
23                                            numeric  
24                                            numeric  
25                                            numeric  
26                                            numeric  
27                                            numeric  
28                                            numeric  
29                                            numeric  
30                                            numeric  
31                                            numeric  
32                                            numeric  
33                                            numeric  
34                                            numeric  
35                                            numeric  
36                                            numeric  
37                                            numeric  
38                                            numeric  
39                                            numeric  
40                                            numeric  
41  {normal,format_clear,ffb_clear,load_clear,perl...

In [7]:
valid_columns = list(df_attributes[0])
df = df[valid_columns]

In [8]:
df.head(10)

duration protocol_type    service   flag  src_bytes  dst_bytes  land  \
0   0.016895        b'tcp'    b'http'  b'SF'      295.0      421.0  b'0'   
1   0.000000        b'udp'  b'domain'  b'S0'       42.0        0.0  b'0'   
2   0.025245        b'tcp'   b'other'  b'SF'     9198.0        0.0  b'0'   
3   0.274966        b'tcp'    b'http'  b'SF'      238.0     3413.0  b'0'   
4   0.060163        b'tcp'    b'http'  b'SF'      231.0    19361.0  b'0'   
5  20.531789        b'tcp'     b'ftp'  b'SF'      223.0      767.0  b'0'   
6   0.043873        b'tcp'    b'http'  b'SF'      319.0     6010.0  b'0'   
7   0.016683        b'tcp'    b'http'  b'SF'      217.0     1321.0  b'0'   
8   0.028091        b'tcp'    b'http'  b'SF'      195.0     3585.0  b'0'   
9   0.021598        b'tcp'    b'http'  b'SF'      354.0     2897.0  b'0'   

   wrong_fragment  urgent  hot  ...  dst_host_srv_count  \
0             0.0     0.0  0.0  ...                96.0   
1             0.0     0.0  0.0  ...                38.0   
2             0.0     0.0  0.0  ...                 0.0   
3             0.0     0.0  0.0  ...                57.0   
4             0.0     0.0  0.0  ...                84.0   
5             0.0     0.0  0.0  ...                 0.0   
6             0.0     0.0  0.0  ...                99.0   
7             0.0     0.0  0.0  ...                96.0   
8             0.0     0.0  0.0  ...                93.0   
9             0.0     0.0  0.0  ...                72.0   

  dst_host_same_srv_rate  dst_host_diff_srv_rate dst_host_same_src_port_rate  \
0                    1.0                     0.0                        0.00   
1                    1.0                     0.0                        0.00   
2                    0.0                     1.0                        0.12   
3                    1.0                     0.0                        0.00   
4                    1.0                     0.0                        0.00   
5                    0.0                     0.0                        0.00   
6                    1.0                     0.0                        0.00   
7                    1.0                     0.0                        0.00   
8                    1.0                     0.0                        0.00   
9                    1.0                     0.0                        0.00   

   dst_host_srv_diff_host_rate  dst_host_serror_rate  \
0                     0.729167              0.000000   
1                     0.394737              0.652174   
2                     0.000000              0.000000   
3                     0.684211              0.111111   
4                     0.547619              0.000000   
5                     0.000000              0.000000   
6                     0.878788              0.000000   
7                     0.937500              0.000000   
8                     0.892473              0.000000   
9                     0.944444              0.000000   

   dst_host_srv_serror_rate  dst_host_rerror_rate  dst_host_srv_error_rate  \
0                  0.000000                   0.0                      0.0   
1                  0.789474                   0.0                      0.0   
2                  0.000000                   0.0                      0.0   
3                  0.087719                   0.0                      0.0   
4                  0.000000                   0.0                      0.0   
5                  0.000000                   0.0                      0.0   
6                  0.000000                   0.0                      0.0   
7                  0.000000                   0.0                      0.0   
8                  0.000000                   0.0                      0.0   
9                  0.000000                   0.0                      0.0   

       class  
0  b'normal'  
1  b'normal'  
2  b'normal'  
3  b'normal'  
4  b'normal'  
5  b'normal'  
6  b'normal'  
7  b'normal'  
8  b'normal'  
9  b'norma

In [9]:
categorical_columns = []

real_value_columns = []

label_col = 'class'
for i,row in df_attributes.iterrows():
    c = row[0]
    t = row[1]
    if t == 'numeric':
        real_value_columns.append(c)
    else:
        if c == label_col:
            continue
        elif t == '{0,1}':
            categorical_columns.append(c)
        else:
            categorical_columns.append(c)
    

In [10]:
categorical_columns

['protocol_type',
 'service',
 'flag',
 'land',
 'logged_in',
 'root_shell',
 'is_hot_login',
 'is_guest_login']

In [11]:
len(real_value_columns)

33

In [12]:
# Convert to string 
for c in list( categorical_columns + [label_col] ):
    df[c] = df[c].str.decode("utf-8")

In [13]:
df = df.dropna()

In [14]:
len(df)

178810

In [15]:

normal_class = ['normal']
df_normal =  df.loc[df['class'].isin(normal_class)]
df_anomalies = df.loc[~df['class'].isin(normal_class)]


In [16]:
len(df_normal),len(df_anomalies)

(174873, 3937)

In [17]:
del df_anomalies['class']
del df_normal['class']

In [18]:
df_anomalies['label'] = 1
df_normal['label'] = 0

/home/ddatta/anaconda3/envs/SG/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/ddatta/anaconda3/envs/SG/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
# Sample 100k rows

_SIZE_ = 100000
df_normal = df_normal.sample(_SIZE_)


In [20]:
master_df = df_normal.append(df_anomalies,ignore_index=True)
master_df = master_df.dropna()

In [21]:
def replace_with_id( row , ref_dict, col):
    value =  row[col]
    if value not in ref_dict.keys():
        row[col] = None
    else:
        row[col] = ref_dict[value]
    return row

In [22]:

single_value_cols = []
target_columns = list(categorical_columns)
entity_count = {}
for i in tqdm(range(len(target_columns))):
    
    column = target_columns[i]
    valid_values = sorted(set(master_df[column]))
    val2id_dict = { 
        e[1]:e[0] for e in enumerate(valid_values,0)
    }
    print(' --> ', column, 'Number of valid values', len(val2id_dict))
    
    if len(val2id_dict) == 1 :
        single_value_cols.append(column)
        #categorical_columns.remove(column)
        continue
        
    entity_count[column] = len(val2id_dict)
        
    master_df = master_df.parallel_apply(
        replace_with_id,
        axis=1,
        args = (val2id_dict, column,)
    )
      

  0%|          | 0/8 [00:00<?, ?it/s]

 -->  protocol_type Number of valid values 3


 12%|█▎        | 1/8 [00:02<00:17,  2.53s/it]

 -->  service Number of valid values 23


 25%|██▌       | 2/8 [00:05<00:15,  2.59s/it]

...

 75%|███████▌  | 6/8 [00:14<00:04,  2.45s/it]

 -->  is_hot_login Number of valid values 2


 88%|████████▊ | 7/8 [00:17<00:02,  2.39s/it]

 -->  is_guest_login Number of valid values 2


100%|██████████| 8/8 [00:19<00:00,  2.43s/it]


In [23]:
categorical_columns

['protocol_type',
 'service',
 'flag',
 'land',
 'logged_in',
 'root_shell',
 'is_hot_login',
 'is_guest_login']

In [24]:
# Normalize the values
def normalize_minmax(value, _max, _min):
    return (value - _min)/(_max -_min)

for column in real_value_columns:
    _min = min(master_df.loc[master_df['label']==0][column])
    _max = max(master_df.loc[master_df['label']==0][column])
    if _max == _min: 
        continue
    master_df[column] = master_df[column].parallel_apply(normalize_minmax, args= (_max,_min, ))
    

master_df = master_df.dropna()
for s in single_value_cols:
    del master_df[s]
    try:
        categorical_columns.remove(s)
    except:
        pass
print(categorical_columns)

['protocol_type', 'service', 'flag', 'land', 'logged_in', 'root_shell', 'is_hot_login', 'is_guest_login']


In [25]:
master_df.columns

Index(['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_hot_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_error_rate', 'label'],
      dtype='object')

In [26]:
# ===========================================
# Keep 2 versions
# 1. one hot encoded
# 2. not one hot enocoded

def create_10_version( df, cat_columns):
    global real_value_columns
    label_Col = 'label'
    df1 = df.copy() 
    print(df1.columns)
    for cc in cat_columns:
        print(cc)
        if entity_count[cc] == 2 :
            _drop_first = True
        else:
            _drop_first = False
        df1 = pd.get_dummies(df1, columns = [cc],drop_first = _drop_first)
    
    all_columns=list(df1.columns)
    disc_columns = [ c for c in all_columns if c != 'label' and c not in real_value_columns]
    ord_cols = disc_columns + real_value_columns + ['label']
    return df1[ord_cols]

In [27]:
master_df_1 = create_10_version( master_df, categorical_columns)

Index(['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_hot_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_error_rate', 'label'],
      dtype='object')
protocol_type
service
flag
land
logged_in
root_shell
is_hot_login
is_guest_login


In [28]:
ordered_columns = categorical_columns + real_value_columns + ['label']
master_df = master_df[ordered_columns]

In [29]:
master_df.columns

Index(['protocol_type', 'service', 'flag', 'land', 'logged_in', 'root_shell',
       'is_hot_login', 'is_guest_login', 'duration', 'src_bytes', 'dst_bytes',
       'wrong_fragment', 'urgent', 'hot', 'num_failed_logins',
       'num_compromised', 'su_attempted', 'num_root', 'num_file_creations',
       'num_shells', 'num_access_files', 'num_outbound_cmds', 'count',
       'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate',
       'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate',
       'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
       'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
       'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
       'dst_host_serror_rate', 'dst_host_srv_serror_rate',
       'dst_host_rerror_rate', 'dst_host_srv_error_rate', 'label'],
      dtype='object')

In [30]:
# Save the files
from pathlib import Path
save_dir = 'processed'
path_obj = Path(save_dir)
path_obj.mkdir(exist_ok=True, parents=True)

# ===========================
# Write out the dimensionality of the columns into a text file
# ============================

col_name_list = []
dimensionality = []
data =[]
for c in categorical_columns:
    col_name_list.append(c)
    v = len(set(master_df[c]))
    dimensionality.append(v)
    data.append((c,v)) 
df_data_dimensions = pd.DataFrame(
    data = data,
    columns=['column','dimension']
)

df_data_dimensions


# Save metadata
f_name = 'data_dimensions.csv'
f_path = os.path.join(save_dir, f_name )
df_data_dimensions.to_csv(f_path,index=False)


utils.save_csv(master_df_1, os.path.join(save_dir,'data_onehot.csv'))
utils.save_csv(master_df, os.path.join(save_dir,'data.csv'))

Size {:.3f} 29.286664962768555  MB 
Size {:.3f} 22.385940551757812  MB 


22.385940551757812

In [31]:
len(master_df_1.columns)

77

In [32]:
ordered_columns

['protocol_type',
 'service',
 'flag',
 'land',
 'logged_in',
 'root_shell',
 'is_hot_login',
 'is_guest_login',
 'duration',
 'src_bytes',
 'dst_bytes',
 'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'num_compromised',
 'su_attempted',
 'num_root',
 'num_file_creations',
 'num_shells',
 'num_access_files',
 'num_outbound_cmds',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_error_rate',
 'label']